# Lab 1


## Part 1: Bilingual dictionary induction and unsupervised embedding-based MT (30%)
*Note: this homework is based on materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [1]:
# !pip install watermark
# DO NOT uncomment this if you are using any 'conda' distribution
# Just run "conda install watermark -c conda-forge" in your terminal

%load_ext watermark

In [2]:
%watermark -v -m -p numpy,gensim,sklearn,nltk -g

CPython 3.7.6
IPython 7.12.0

numpy 1.18.1
gensim 3.8.0
sklearn 0.22.1
nltk 3.4.5

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-64-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 48
interpreter: 64bit
Git hash   : 5827d18df161965f5fdf072702f7a474bdf684c3


In [3]:
from os import remove
from pickle import dump, load
from functools import partial
from operator import itemgetter
from csv import reader as csv_reader

from typing import Sequence, Tuple

import gensim
import numpy as np

from gensim.models import KeyedVectors
from sklearn.linear_model import LinearRegression
from nltk import word_tokenize


tsv_reader = partial(csv_reader, delimiter='\t')

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages. Please use word2vec-compatible format (.text).

In [4]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.vec.gz
# !gzip -d cc.uk.300.vec.gz cc.ru.300.vec.gz

In [5]:
# %%time
# uk_emb = KeyedVectors.load_word2vec_format('cc.uk.300.vec')
# ru_emb = KeyedVectors.load_word2vec_format('cc.ru.300.vec')

In [6]:
# %%time
# with open('rus_300_emb.pkl', 'wb') as out:
#     dump(ru_emb, out)
# with open('ukr_300_emb.pkl', 'wb') as out:
#     dump(uk_emb, out)
# # remove('cc.ru.300.vec')
# # remove('cc.uk.300.vec')

In [7]:
%%time
with open('rus_300_emb.pkl', 'rb') as pkl:
    ru_emb = load(pkl)
with open('ukr_300_emb.pkl', 'rb') as pkl:
    uk_emb = load(pkl)

CPU times: user 8.74 s, sys: 7.28 s, total: 16 s
Wall time: 15.9 s


In [8]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.893003523349762),
 ('апрель', 0.8729087710380554),
 ('декабрь', 0.8652557134628296),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401416540145874)]

In [9]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 1.0),
 ('липень', 0.9096440076828003),
 ('вересень', 0.9016969799995422),
 ('червень', 0.8992520570755005),
 ('жовтень', 0.8810407519340515),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431221961975)]

In [10]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.27579623460769653),
 ('ЖИЗНИВадим', 0.25203439593315125),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829229712486267),
 ('Ведущий-Алексей', 0.24438698589801788),
 ('Недопустимость', 0.24435283243656158),
 ('2Михаил', 0.23981399834156036),
 ('лексей', 0.23740756511688232),
 ('комплексн', 0.23695148527622223),
 ('персональ', 0.2368222326040268)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [11]:
def load_word_pairs(filename):

    with open(filename, 'r') as inpf:
        uk_ru_pairs, uk_vectors, ru_vectors = zip(
            *(
                ((uk, ru), uk_emb[uk], ru_emb[ru])
                for (uk, ru) in tsv_reader(inpf)
                if uk in uk_emb and ru in ru_emb
            )
        )
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [12]:
# !wget -O ukr_rus_train.tsv http://tiny.cc/jfgecz
# !wget -O ukr_rus_test.tsv http://tiny.cc/6zoeez

In [13]:
!head ukr_rus_test.tsv

або	либо
активний	активный
актор	актер
але	ж
асамблея	собрание
бабуся	бабушка
багажник	ствол
бажати	желать
башта	башня
бізнес	бизнес


In [14]:
uk_ru_train, X_train, Y_train = load_word_pairs('ukr_rus_train.tsv')

In [15]:
uk_ru_test, X_test, Y_test = load_word_pairs('ukr_rus_test.tsv')

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [16]:
mapping = LinearRegression(fit_intercept=False, n_jobs=-1)
mapping.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=-1, normalize=False)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [17]:
august = mapping.predict(uk_emb['серпень'].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531432151794434),
 ('июнь', 0.8402522206306458),
 ('март', 0.8385882377624512),
 ('сентябрь', 0.8331484198570251),
 ('февраль', 0.8311208486557007),
 ('октябрь', 0.8278017640113831),
 ('ноябрь', 0.8243728280067444),
 ('июль', 0.822961688041687),
 ('август', 0.8112279772758484),
 ('январь', 0.8022985458374023)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [18]:
def precision(pairs:          Sequence[Tuple[str, str]],
              mapped_vectors: Sequence[Sequence[float]],
              topn:           int = 1) -> float:
    """
    :param pairs:           list of right word pairs [(uk_word_0, ru_word_0), ...]
    :param mapped_vectors:  list of embeddings after mapping
                            from source embedding space to destination embedding space
    :param topn:            the number of nearest neighbours in destination embedding space to choose from

    :return:                precision_val
                            fraction of total number of words for which we can find right translation at top K.
    """
    if len(pairs) != len(mapped_vectors):
        raise AssertionError("Parameter 'pairs' should have the same length as parameter 'mapped_vectors'")

    num_matches = sum(
        true_rus_tran in map(itemgetter(0), ru_emb.most_similar(emb.reshape(1, -1), topn=topn))
        for emb, true_rus_tran in zip(mapped_vectors, map(itemgetter(1), pairs))
    )
    precision_val = num_matches / len(pairs)
    return precision_val

In [19]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [20]:
%%time
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

CPU times: user 11min 3s, sys: 7 s, total: 11min 10s
Wall time: 27.9 s


In [21]:
%%time
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

CPU times: user 12min 35s, sys: 8.21 s, total: 12min 43s
Wall time: 31.8 s


In [22]:
print(precision_top1)
print(precision_top5)

0.628498727735369
0.7913486005089059


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [23]:
def learn_transform(X_train: np.ndarray, Y_train: np.ndarray) -> np.ndarray:
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """

    u, s, vh = np.linalg.svd(X_train.T @ Y_train)
    del s
    return u @ vh

In [24]:
W = learn_transform(X_train, Y_train)

In [25]:
ru_emb.most_similar([uk_emb["серпень"] @ W])

[('апрель', 0.8245130777359009),
 ('июнь', 0.8056631088256836),
 ('сентябрь', 0.8055763244628906),
 ('март', 0.8032934069633484),
 ('октябрь', 0.7987103462219238),
 ('июль', 0.7946797609329224),
 ('ноябрь', 0.7939637303352356),
 ('август', 0.7938190698623657),
 ('февраль', 0.7923860549926758),
 ('декабрь', 0.771537721157074)]

In [26]:
%%time
print(precision(uk_ru_test, X_test @ W))
print(precision(uk_ru_test, X_test @ W, 5))

0.6437659033078881
0.7989821882951654
CPU times: user 11min 34s, sys: 7.19 s, total: 11min 41s
Wall time: 29.3 s


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [27]:
# !wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz && gzip -d uk.txt.gz

In [28]:
with open('uk.txt', 'r') as f:
    uk_corpus = f.readlines()

uk_corpus = uk_corpus[:1000]

In [29]:
vocab = uk_emb.vocab
unk_embed = np.zeros(300, dtype=np.float32)


def translate(sentence: str, conf_thresh: float = 0.4) -> str:
    """
    :param sentence:     sentence in Ukrainian (str)
    :param conf_thresh:  confidence level above which
                         the program replaces the Ukrainian word with the Russian translation candidate

    :return:             translation - sentence in Russian (str)

    * finds ukrainian embedding for each word in sentence
    * transforms ukrainian embedding vector
    * finds nearest russian word and replace
    """

    sentence = word_tokenize(sentence)
    sentence_embed = np.array(
        tuple(
            map(
                lambda word: uk_emb[word] if word in vocab else unk_embed,
                sentence
            )
        )
    )
    sentence_embed = sentence_embed @ W


    tran_generator = (
        (word_tran if confidence > conf_thresh else original_word)
        for ((word_tran, confidence), original_word) in zip(
            map(
                lambda word_embed: ru_emb.most_similar(word_embed.reshape(1, -1), topn=1)[0],
                sentence_embed
            ),
            sentence
        )
    )

    return ' '.join(tran_generator)

In [30]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [31]:
%%time
for sent in uk_corpus[::10]:
    print(translate(sent))
print()

Я уже закончу колледж , когда мы прибежишь со Америки .
Город бомбили враждебные самолеты .
Возможно , мной антисоциальный , конечно это не означает , что мной не общаюсь со людьми .
Впрочем утра выпала роса .
Беда не приходит одна .
Посмотри по тот дым .
Я заказал два гамбургера .
Я не хотел никого обидеть .
Гора покрыта снегом .
по фотографии во девушки корона не со золота , а со цветов .
Во меня є мечта .
Я приехал во Японию со Китая .
по север находится Шотландия ; по юге — Англия ; по востоке — Уэльс ; и ещe дальше по востоке — северная Ирландия .
Его родная страна — Германия .
Берн — столица Швейцарии .
Он ждал по него к десятой часа .
Ты можешь взять ту книгу даром .
Такой роман сочинил известный американский писатель .
забронировать , будте ласковые , комнату возле международного аэропорта во Торонто .
Он знает , что ты его влюбится ?
Я знаю , что ты богатый .
Те , кто всё забывают , счастливые .
Во этой реке опасно плавать .
пришел , увидел , победил .
Я хожу к школы пешком .


Great! 
See second notebook for the Neural Machine Translation assignment.